In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional, Input, Reshape # type: ignore
from tensorflow.keras.optimizers import Adam, RMSprop  # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau # type: ignore
from tensorflow.keras.regularizers import l2 # type: ignore


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(f"TensorFlow Version: {tf.__version__}")

# Load the .mat file
file_path = '../../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']
data = EV_data
del EV_data
print(data.shape)

2024-05-29 12:49:45.243112: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow Version: 2.16.1
(50, 1000, 832)


In [2]:
# Function to create sequences
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(data.shape[0]):  # iterate over samples
        for j in range(data.shape[1] - n_steps_in - n_steps_out + 1):  # iterate over timesteps
            seq_x = data[i, j:j + n_steps_in]
            seq_y = data[i, j + n_steps_in:j + n_steps_in + n_steps_out]
            X.append(seq_x)
            y.append(seq_y)
    return np.array(X), np.array(y)


timesteps_in = 10

X, y = create_sequences(data, timesteps_in,5)
print(f'X shape: {X.shape}, y shape: {y.shape}')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X shape: (49300, 10, 832), y shape: (49300, 5, 832)


# Preprocessing

In [3]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((39440, 10, 832), (9860, 10, 832), (39440, 5, 832), (9860, 5, 832))

In [4]:
# Define a function to create the advanced LSTM model with gradient clipping
def create_model(dropout_rate=0.2, lstm_units=256, dense_units=512):
    model = Sequential()
    model.add(Input(shape=(timesteps_in, 832)))
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(lstm_units)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(dense_units, activation='elu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(832*5, activation='elu'))
    model.add(Reshape((5,832)))
    
    optimizer = Adam(clipvalue=1.0)

    model.compile(optimizer=optimizer, loss='mse')
    return model


# gelu
# 309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0126
# Test Loss: 0.012605997733771801
# 309/309 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
# Mean Squared Error (MSE): 0.012605994747627473
# Mean Absolute Error (MAE): 0.0924165528135168
# R-squared (R^2): 0.1767418719445527
# 124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0132 - val_loss: 0.0126 - learning_rate: 7.8125e-06


# relu
# 309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0141
# Test Loss: 0.014148660935461521
# 309/309 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
# Mean Squared Error (MSE): 0.014148659255275984
# Mean Absolute Error (MAE): 0.09911822359734364
# R-squared (R^2): 0.0708324295659552
# 124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0145 - val_loss: 0.0142 - learning_rate: 1.0000e-06



# # elu
# 309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0124
# Test Loss: 0.012479486875236034
# 309/309 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
# Mean Squared Error (MSE): 0.01247947787391801
# Mean Absolute Error (MAE): 0.09178420758552314
# R-squared (R^2): 0.18487061241083902
# 124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0131 - val_loss: 0.0125 - learning_rate: 1.0000e-06



In [5]:

best_params = {

    'dropout_rate': 0.2,
    'lstm_units': 1024,
    'dense_units': 1024,
    'batch_size': 256,
    'epochs': 250,
}

# Create and train the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

model = create_model(
    dropout_rate=best_params['dropout_rate'],
    lstm_units=best_params['lstm_units'],
    dense_units=best_params['dense_units'],
)



2024-05-29 12:49:53.169692: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-29 12:49:53.169731: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: federatedstuff
2024-05-29 12:49:53.169739: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: federatedstuff
2024-05-29 12:49:53.169828: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.54.15
2024-05-29 12:49:53.169854: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  550.54.15  Release Build  (dvs-builder@U16-A24-23-2)  Tue Mar  5 22:15:33 UTC 2024
GCC version:  gcc version 12.3.0 (Ubuntu 12.3.0-1ubuntu1~22.04) 
"


In [6]:

history = model.fit(
    X_train, y_train,
    batch_size=best_params['batch_size'],
    epochs=best_params['epochs'],
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1)

2024-05-29 12:49:58.968725: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1050050560 exceeds 10% of free system memory.


Epoch 1/250


2024-05-29 12:50:00.236191: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 525025280 exceeds 10% of free system memory.


 61/124 ━━━━━━━━━━━━━━━━━━━━ 2:56 3s/step - loss: 0.1202

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")

predictions = model.predict(X_test)

# Flatten y_test and predictions along the first axis
y_test_flat = y_test.reshape(-1, y_test.shape[-1])
predictions_flat = predictions.reshape(-1, predictions.shape[-1])

# Calculate metrics
mse = mean_squared_error(y_test_flat, predictions_flat)
mae = mean_absolute_error(y_test_flat, predictions_flat)
r2 = r2_score(y_test_flat, predictions_flat)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R^2): {r2}')

309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0124
Test Loss: 0.012479486875236034
309/309 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Mean Squared Error (MSE): 0.01247947787391801
Mean Absolute Error (MAE): 0.09178420758552314
R-squared (R^2): 0.18487061241083902


In [ ]:
import pandas as pd
ue = 100
ts = 1
pd.DataFrame(predictions[ue][ts],y_test[ue][ts])

,0
-0.119735,-0.026965
0.088507,-0.012636
0.167101,0.059234
-0.147060,-0.073452
0.073966,-0.043460
...,...
-0.163436,-0.064524
-0.107874,0.040137
0.058618,-0.048976
-0.078371,-0.078766
